<a href="https://colab.research.google.com/github/A-Wadhwani/ME498-Project/blob/main/08_Boosting_Ensemble_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Open drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import tensorflow as tf
import pandas as pd
import random
import time
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.metrics import r2_score, mean_squared_log_error, mean_squared_error

In [ ]:
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers

This is an ensemble model from various Boosting and Averaging ensemble models:
- XGBoost
- Catboost
- Random Forest
- Extra Trees

A Stacking Regressor is then used to combine these models with Gradient Boosted Decision Trees as a final estimator

In [ ]:
!pip install catboost

In [ ]:
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, ExtraTreesRegressor
import xgboost as xgb
import catboost as cb
from sklearn.svm import SVR

In [ ]:
# Reading training and testing dataset files
path = "/kaggle/input/ashraeiii/"
x_train = pd.read_csv(path + "x_train.csv", index_col=0)
x_test = pd.read_csv(path + "x_test.csv", index_col=0)
y_train = pd.read_csv(path + "y_train.csv", index_col=0)
y_test = pd.read_csv(path + "y_test.csv", index_col=0)

/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
models = {}

In [ ]:
cb_params = {
        'n_estimators': 10000,
        'learning_rate': 0.25,
        'eval_metric': 'RMSE',
        'loss_function': 'RMSE',
        'metric_period': 10,
        'task_type': 'GPU',
        'depth': 14,
}
cb_model = cb.CatBoostRegressor(**cb_params)
models['catboost'] = cb_model
start_time = time.time()
cb_model.fit(x_train, y_train, eval_set=(x_test, y_test), use_best_model=True, verbose=100, early_stopping_rounds=100)
start_pred = time.time()
y_predict = cb_model.predict(x_test)
print("Training Time: ", start_pred - start_time)
print("Prediction Time: ", time.time() - start_pred)
print("MSE: ", mean_squared_error(y_test, y_predict))
print("R2 score: ", r2_score(y_test, y_predict))

0:	learn: 229.0094245	test: 227.8561283	best: 227.8561283 (0)	total: 34.1ms	remaining: 2m 50s
100:	learn: 44.4122893	test: 45.4599897	best: 45.4599897 (100)	total: 1.8s	remaining: 1m 27s
200:	learn: 38.4691165	test: 40.4350181	best: 40.4350181 (200)	total: 3.46s	remaining: 1m 22s
300:	learn: 36.3435287	test: 38.8677072	best: 38.8654595 (293)	total: 5.28s	remaining: 1m 22s
400:	learn: 34.1880410	test: 37.3751347	best: 37.3750838 (399)	total: 7.1s	remaining: 1m 21s
500:	learn: 32.2165335	test: 36.0580597	best: 36.0580597 (500)	total: 9.04s	remaining: 1m 21s
600:	learn: 31.0987184	test: 35.4342916	best: 35.4342629 (599)	total: 11.2s	remaining: 1m 22s
700:	learn: 30.0223484	test: 34.7587147	best: 34.7587147 (700)	total: 13.1s	remaining: 1m 20s
800:	learn: 28.9233893	test: 34.0692584	best: 34.0692509 (799)	total: 15.5s	remaining: 1m 21s
900:	learn: 27.9878344	test: 33.5858321	best: 33.5858321 (900)	total: 17.3s	remaining: 1m 18s
1000:	learn: 27.2113738	test: 33.2466831	best: 33.2466831 (100

In [ ]:
ext_model = ExtraTreesRegressor(n_estimators=100, criterion='mse', max_depth=14)
start_time = time.time()
ext_model.fit(x_train, y_train.values.ravel())
models['extra_trees'] = ext_model
start_pred = time.time()
y_predict = ext_model.predict(x_test)
print("Training Time: ", start_pred - start_time)
print("Prediction Time: ", time.time() - start_pred)
print("MSE: ", mean_squared_error(y_test, y_predict))
print("R2 score: ", r2_score(y_test, y_predict))

Training Time:  286.19740748405457
Prediction Time:  7.7983198165893555
MSE:  1758.1004169827818
R2 score:  0.9790826039614726


In [ ]:
rf_model = RandomForestRegressor(n_estimators=25)
start_time = time.time()
rf_model.fit(x_train, y_train.values.ravel())
models['random_forest'] = rf_model
start_pred = time.time()
y_predict = rf_model.predict(x_test)
print("Training Time: ", start_pred - start_time)
print("Prediction Time: ", time.time() - start_pred)
print("MSE: ", mean_squared_error(y_test, y_predict))
print("R2 score: ", r2_score(y_test, y_predict))

Training Time:  227.50843286514282
Prediction Time:  7.369952201843262
MSE:  499.6026619834934
R2 score:  0.9940558647039366


In [ ]:
xgb_model = xgb.XGBRegressor(tree_method='gpu_hist', n_estimators=1000, max_depth=14,learning_rate=0.01, verbose=1)
start_time = time.time()
xgb_model.fit(x_train, y_train.values.ravel())
models['xgboost'] = xgb_model
start_pred = time.time()
y_predict =  xgb_model.predict(x_test)
print("Training Time: ", start_pred - start_time)
print("Prediction Time: ", time.time() - start_pred)
print("MSE: ", mean_squared_error(y_test, y_predict))
print("R2 score: ", r2_score(y_test, y_predict))

[19:40:42] WARNING: ../src/learner.cc:573: 
Parameters: { "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


Training Time:  328.448379278183
Prediction Time:  90.139657497406
MSE:  521.4802500628552
R2 score:  0.9937955711679114


## Model Evaluation

In [ ]:
results = pd.DataFrame()
results['catboost'] = cb_model.predict(x_train)
results['extra_trees'] = ext_model.predict(x_train)
results['random_forest'] = rf_model.predict(x_train)
results['xgboost'] = xgb_model.predict(x_train)
results.head()

/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


,catboost,extra_trees,random_forest,xgboost
0,273.793888,261.852022,265.55752,267.435852
1,67.011890,68.426718,79.50000,76.706276
2,0.878116,0.002596,0.00000,-0.557095
3,57.593034,54.379789,63.62470,60.537224
4,133.154918,128.994951,137.86240,126.273727


In [ ]:
from sklearn.linear_model import LinearRegression
ensemble_model = LinearRegression()
start_time = time.time()
ensemble_model.fit(results, y_train)
print("Training Time: ", time.time() - start_time)
print("Coefficients: ", ensemble_model.coef_)

Training Time:  0.10887670516967773
Coefficients:  [[ 0.22457277 -0.09436597  0.84677442  0.02173296]]


In [ ]:
ens_test = pd.DataFrame()
start_time = time.time()
ens_test['catboost'] = cb_model.predict(x_test)
ens_test['extra_trees'] = ext_model.predict(x_test)
ens_test['random_forest'] = rf_model.predict(x_test)
ens_test['xgboost'] = xgb_model.predict(x_test)
ens_test.head()

/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


,catboost,extra_trees,random_forest,xgboost
0,40.050714,70.220978,49.8432,58.433769
1,17.075296,20.069947,17.3440,18.809311
2,289.607965,258.974614,290.1344,279.104889
3,9.578188,7.527609,5.8320,4.536321
4,-3.785544,0.008385,0.0000,-0.395855


In [ ]:
y_predict = ensemble_model.predict(ens_test)
print("MSE: ", mean_squared_error(y_test, y_predict))
print("R2 score: ", r2_score(y_test, y_predict))
print("Prediction Time: ", time.time() - start_time)

MSE:  471.3745446728544
R2 score:  0.9943917150930867
Prediction Time:  115.7962064743042
